In [3]:

import warnings
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


%matplotlib inline
import os 

import re
import time



import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU

import datetime

%matplotlib inline
pd.set_option("max_columns", None)

import mplleaflet

import geopandas as gpd

import shapely

import seaborn as sns

import pysal as ps

from geopandas import GeoDataFrame
from shapely.geometry import Point

from scipy.stats.mstats import mode

In [4]:
#speed data with lat and lon added
df1 = pd.read_csv ("speedClean.csv") 

In [345]:

df1["avSpeed"] = (df1["lowSpeed"]+df1["highSpeed"])/2

In [346]:

##also look at period matching accident data, mid 2014 - mid 2016

df2 = df1.loc[(df1["timestamp"] > "2014-6-1") & (df1["timestamp"] < "2016-6-22")]

In [347]:
## timestamps for total df1 range from 7-2012 to 6-2016, there are 54000 or so readings
## timestamps for df2 are oct 2015 to june 2016

In [348]:
print(max(df1["timestamp"]),min(df1["timestamp"]),len(df1["timestamp"]))

2016-06-22 23:00:00 2012-07-31 00:00:00 54340


In [349]:
print(max(df2["timestamp"]),min(df2["timestamp"]),len(df2["timestamp"]))

2016-06-22 23:00:00 2015-10-06 12:00:00 12220


NameError: name 'df2' is not defined

In [350]:
## there are speed reports from these 9 neighborhoods
print(pd.unique(df1["nhood"]), len(pd.unique(df1["nhood"])))

['SOUTH BOSTON' 'CHARLESTOWN' 'DORCHESTER' 'WEST ROXBURY' 'ROXBURY'
 'ALLSTON BRIGHTON' 'HYDE PARK' 'EAST BOSTON' 'BOSTON PROPER'] 9


In [351]:
print(pd.unique(df2["nhood"]), len(pd.unique(df2["nhood"])))

['DORCHESTER' 'WEST ROXBURY' 'ROXBURY' 'SOUTH BOSTON' 'ALLSTON BRIGHTON'
 'BOSTON PROPER'] 6


In [352]:
#west rox has had the most study_ids assigned, and the highest number of readings
df1["study_id"].groupby(df1['nhood']).nunique().sort_values(ascending=False)




nhood
WEST ROXBURY        17
DORCHESTER          15
ROXBURY             12
SOUTH BOSTON         5
CHARLESTOWN          3
ALLSTON BRIGHTON     3
HYDE PARK            2
EAST BOSTON          1
BOSTON PROPER        1
Name: study_id, dtype: int64

In [354]:
df1['nhood'].value_counts() # this many readings per neigborhood

WEST ROXBURY        17160
DORCHESTER          14404
SOUTH BOSTON         7800
ROXBURY              5616
ALLSTON BRIGHTON     3432
HYDE PARK            2808
CHARLESTOWN          1560
EAST BOSTON           936
BOSTON PROPER         624
Name: nhood, dtype: int64

In [355]:
#rox has had the most study_ids assigned
df2["study_id"].groupby(df2['nhood']).nunique().sort_values(ascending=False)



nhood
ROXBURY             6
WEST ROXBURY        3
DORCHESTER          3
SOUTH BOSTON        2
ALLSTON BRIGHTON    2
BOSTON PROPER       1
Name: study_id, dtype: int64

In [356]:
df2['nhood'].value_counts() # this many readings per neigborhood

ALLSTON BRIGHTON    3120
ROXBURY             3120
DORCHESTER          2548
WEST ROXBURY        1560
SOUTH BOSTON        1248
BOSTON PROPER        624
Name: nhood, dtype: int64

In [357]:
#av reading per studyid per nhood
df1['nhood'].value_counts()/df1["study_id"].groupby(df1['nhood']).nunique().sort_values(ascending=False)




ALLSTON BRIGHTON    1144.000000
BOSTON PROPER        624.000000
CHARLESTOWN          520.000000
DORCHESTER           960.266667
EAST BOSTON          936.000000
HYDE PARK           1404.000000
ROXBURY              468.000000
SOUTH BOSTON        1560.000000
WEST ROXBURY        1009.411765
dtype: float64

In [358]:

#av reading per study id per nhood
df2['nhood'].value_counts()/df2["study_id"].groupby(df2['nhood']).nunique().sort_values(ascending=False)



ALLSTON BRIGHTON    1560.000000
BOSTON PROPER        624.000000
DORCHESTER           849.333333
ROXBURY              520.000000
SOUTH BOSTON         624.000000
WEST ROXBURY         520.000000
dtype: float64

In [359]:
## average speed per neighborhood

In [362]:
df1['avSpeed'].groupby(df1['nhood']).mean().sort_values(ascending=False)

nhood
BOSTON PROPER       41.500000
ALLSTON BRIGHTON    40.706294
ROXBURY             37.619658
DORCHESTER          34.313663
SOUTH BOSTON        34.166154
WEST ROXBURY        33.562937
HYDE PARK           32.769231
EAST BOSTON         32.769231
CHARLESTOWN         32.769231
Name: avSpeed, dtype: float64

In [363]:
df2['avSpeed'].groupby(df2['nhood']).mean().sort_values(ascending=False)

nhood
WEST ROXBURY        41.5
SOUTH BOSTON        41.5
ROXBURY             41.5
DORCHESTER          41.5
BOSTON PROPER       41.5
ALLSTON BRIGHTON    41.5
Name: avSpeed, dtype: float64

## number of speed readings per neighborhood
* boston proper has high average speeds but the least amount of readings

In [369]:
#number of readings per study id per neighborhood, looks like dorchester has the most study_ids
df1["study_id"].groupby(df1['nhood']).value_counts()

nhood             study_id
ALLSTON BRIGHTON  8315        2496
                  8360         624
                  7171         312
BOSTON PROPER     8387         624
CHARLESTOWN       7219         624
                  7237         624
                  7132         312
DORCHESTER        7168        1872
                  7243        1872
                  7255        1872
                  7912        1196
                  8038        1196
                  7135         936
                  7144         936
                  7147         936
                  7156         936
                  7138         624
                  7159         624
                  7222         624
                  7228         312
                  7249         312
                  8039         156
EAST BOSTON       7458         936
HYDE PARK         7207        1872
                  7204         936
ROXBURY           7980        1248
                  7987        1248
                  7186      

In [370]:
df2["study_id"].groupby(df2['nhood']).value_counts()

nhood             study_id
ALLSTON BRIGHTON  8315        2496
                  8360         624
BOSTON PROPER     8387         624
DORCHESTER        7912        1196
                  8038        1196
                  8039         156
ROXBURY           7980        1248
                  7987        1248
                  7978         156
                  7981         156
                  7984         156
                  7985         156
SOUTH BOSTON      8296         624
                  8350         624
WEST ROXBURY      7976        1248
                  7970         156
                  7972         156
Name: study_id, dtype: int64

In [371]:
# av speed per study_id ranked, there are dif numbers of study ids per nhood though..
df1['avSpeed'].groupby([df1['study_id'],df1['nhood']]).mean().sort_values(ascending=False).head(20)


study_id  nhood           
8387      BOSTON PROPER       41.500000
7984      ROXBURY             41.500000
8360      ALLSTON BRIGHTON    41.500000
7912      DORCHESTER          41.500000
7970      WEST ROXBURY        41.500000
7972      WEST ROXBURY        41.500000
7978      ROXBURY             41.500000
7980      ROXBURY             41.500000
7981      ROXBURY             41.500000
7976      WEST ROXBURY        41.500000
7985      ROXBURY             41.500000
7987      ROXBURY             41.500000
8038      DORCHESTER          41.500000
8039      DORCHESTER          41.500000
8296      SOUTH BOSTON        41.500000
8315      ALLSTON BRIGHTON    41.500000
8350      SOUTH BOSTON        41.500000
7162      WEST ROXBURY        32.769231
7189      ROXBURY             32.769231
7186      ROXBURY             32.769231
Name: avSpeed, dtype: float64

In [372]:
df2['avSpeed'].groupby([df2['study_id'],df2['nhood']]).mean().sort_values(ascending=False).head(20)



study_id  nhood           
8387      BOSTON PROPER       41.5
7984      ROXBURY             41.5
7970      WEST ROXBURY        41.5
7972      WEST ROXBURY        41.5
7976      WEST ROXBURY        41.5
7978      ROXBURY             41.5
7980      ROXBURY             41.5
7981      ROXBURY             41.5
7985      ROXBURY             41.5
8360      ALLSTON BRIGHTON    41.5
7987      ROXBURY             41.5
8038      DORCHESTER          41.5
8039      DORCHESTER          41.5
8296      SOUTH BOSTON        41.5
8315      ALLSTON BRIGHTON    41.5
8350      SOUTH BOSTON        41.5
7912      DORCHESTER          41.5
Name: avSpeed, dtype: float64

In [373]:
# slowest average speeds by study_id

In [374]:
df1['avSpeed'].groupby([df1['study_id'],df1['nhood']]).mean().sort_values(ascending=False).tail(20)



study_id  nhood       
7237      CHARLESTOWN     32.769231
7462      WEST ROXBURY    32.769231
7458      EAST BOSTON     32.769231
7255      DORCHESTER      32.769231
7252      ROXBURY         32.769231
7249      DORCHESTER      32.769231
7246      WEST ROXBURY    32.769231
7243      DORCHESTER      32.769231
7240      WEST ROXBURY    32.769231
7234      WEST ROXBURY    32.769231
7201      WEST ROXBURY    32.769231
7231      WEST ROXBURY    32.769231
7228      DORCHESTER      32.769231
7219      CHARLESTOWN     32.769231
7216      WEST ROXBURY    32.769231
7213      SOUTH BOSTON    32.769231
7210      SOUTH BOSTON    32.769231
7207      HYDE PARK       32.769231
7204      HYDE PARK       32.769231
7129      SOUTH BOSTON    32.769231
Name: avSpeed, dtype: float64

In [375]:
df2['avSpeed'].groupby([df2['study_id'],df2['nhood']]).mean().sort_values(ascending=False).tail(20)



study_id  nhood           
8387      BOSTON PROPER       41.5
7984      ROXBURY             41.5
7970      WEST ROXBURY        41.5
7972      WEST ROXBURY        41.5
7976      WEST ROXBURY        41.5
7978      ROXBURY             41.5
7980      ROXBURY             41.5
7981      ROXBURY             41.5
7985      ROXBURY             41.5
8360      ALLSTON BRIGHTON    41.5
7987      ROXBURY             41.5
8038      DORCHESTER          41.5
8039      DORCHESTER          41.5
8296      SOUTH BOSTON        41.5
8315      ALLSTON BRIGHTON    41.5
8350      SOUTH BOSTON        41.5
7912      DORCHESTER          41.5
Name: avSpeed, dtype: float64

In [376]:
#these are the study ids ranked by avSpeed of avSpeed speeed per car
#df22.sort_values(ascending=False).head()
df1['avSpeed'].groupby(df1['nhood']).describe()

nhood                  
ALLSTON BRIGHTON  count     3432.000000
                  mean        40.706294
                  std         19.161526
                  min          3.000000
                  25%         27.000000
                  50%         42.000000
                  75%         57.000000
                  max         70.000000
BOSTON PROPER     count      624.000000
                  mean        41.500000
                  std         19.071884
                  min          7.500000
                  25%         27.000000
                  50%         42.000000
                  75%         57.000000
                  max         70.000000
CHARLESTOWN       count     1560.000000
                  mean        32.769231
                  std         18.357803
                  min          3.000000
                  25%         18.000000
                  50%         33.000000
                  75%         48.000000
                  max         60.000000
DORCHESTER      

In [377]:
df2['avSpeed'].groupby(df2['nhood']).describe()

nhood                  
ALLSTON BRIGHTON  count    3120.000000
                  mean       41.500000
                  std        19.059651
                  min         7.500000
                  25%        27.000000
                  50%        42.000000
                  75%        57.000000
                  max        70.000000
BOSTON PROPER     count     624.000000
                  mean       41.500000
                  std        19.071884
                  min         7.500000
                  25%        27.000000
                  50%        42.000000
                  75%        57.000000
                  max        70.000000
DORCHESTER        count    2548.000000
                  mean       41.500000
                  std        19.060336
                  min         7.500000
                  25%        27.000000
                  50%        42.000000
                  75%        57.000000
                  max        70.000000
ROXBURY           count    3120.000000
 

In [379]:

## how to find the modeimport numpy as np
from scipy import stats
from scipy.stats import mode




In [380]:
#just speeds that are over 55
over55df1 = df1[(df1['avSpeed']>55)]
over55df2 = df2[(df2['avSpeed']>55)]

In [381]:
#count of number of times those speeds clocked by study id
over55df1['avSpeed'].groupby([over55df1['study_id'],over55df1['nhood']]).count().sort_values(ascending=False).head()


study_id  nhood           
8315      ALLSTON BRIGHTON    768
7210      SOUTH BOSTON        576
7976      WEST ROXBURY        384
7987      ROXBURY             384
7980      ROXBURY             384
Name: avSpeed, dtype: int64

In [382]:
over55df2['avSpeed'].groupby([over55df2['study_id'],over55df2['nhood']]).count().sort_values(ascending=False).head()



study_id  nhood           
8315      ALLSTON BRIGHTON    768
7987      ROXBURY             384
7976      WEST ROXBURY        384
7980      ROXBURY             384
7912      DORCHESTER          368
Name: avSpeed, dtype: int64

In [383]:
#count of number of times those speeds clocked by nhood, but some nhoods, as a percentage
g = over55df1['avSpeed'].groupby(over55df1['nhood']).count()



In [385]:
g.sort_values(ascending=False).head()




nhood
WEST ROXBURY        2880
DORCHESTER          2608
SOUTH BOSTON        1392
ROXBURY             1344
ALLSTON BRIGHTON    1008
Name: avSpeed, dtype: int64

In [333]:
#percentage of counts of avspeed over 55 vs total counts of all speeds per study_id per nhood
#using full data set

In [386]:
a = over55df1['avSpeed'].groupby([over55df1['study_id'],over55df1['nhood']]).count()
b = df1['avSpeed'].groupby([df1['study_id'],df1['nhood']]).count()

In [388]:
c = a/b
c.sort_values(ascending=False).head()



study_id  nhood           
8387      BOSTON PROPER       0.307692
7984      ROXBURY             0.307692
8360      ALLSTON BRIGHTON    0.307692
7912      DORCHESTER          0.307692
7970      WEST ROXBURY        0.307692
Name: avSpeed, dtype: float64

In [390]:
h = over55df2['avSpeed'].groupby([over55df2['study_id'],over55df2['nhood']]).count()


In [391]:
i = h/b
i.sort_values(ascending=False).head()



study_id  nhood        
8387      BOSTON PROPER    0.307692
7984      ROXBURY          0.307692
7970      WEST ROXBURY     0.307692
7972      WEST ROXBURY     0.307692
7976      WEST ROXBURY     0.307692
Name: avSpeed, dtype: float64